# Milestone 2 : RADI project

This is the notebook file where we gather our data, preprocess some of it and start some of the analysis

In [1]:
# Standard imports. Tweepy is a library to interact easily with the Twitter API via Python.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
import tweepy 
import csv
import http.client
import tokenize

## Election Data

In this part, we load and clean the data regarding the elections. 

In [2]:
PATH_ELECTIONS_2015 = "data/elections/elections_2015.csv"
PATH_ELECTIONS_2019 = "data/elections/elections_2019.csv"

romand_cantons = ['Vaud','Jura','Genève','Valais / Wallis','Neuchâtel','Fribourg / Freiburg']
selected_parties = ['SVP','SP','FDP','CVP','GPS','GLP']
columns_required = {'wahl_jahr':'year','kanton_bezeichnung':'canton','partei_staerke':'party strength',
                    'partei_bezeichnung_de': "party's name in German",
                    'partei_bezeichnung_fr': "party's name in French",
                    'partei_bezeichnung_en': "party's name in English",
                    'fiktive_waehlende': 'fictional voter'}

# To understand the concept of fictional voter: https://en.wikipedia.org/wiki/National_Council_(Switzerland)

In [3]:
elections_2015 = pd.read_csv(PATH_ELECTIONS_2015, sep=';')
elections_2015

,wahl_jahr,kanton_nummer,kanton_bezeichnung,partei_id,partei_bezeichnung_de,partei_bezeichnung_fr,partei_bezeichnung_it,partei_bezeichnung_en,anzahl_listen,anzahl_kandidierende,...,letzte_wahl_anzahl_gewaehlte_f,letzte_wahl_anzahl_gewaehlte_m,differenz_anzahl_gewaehlte,differenz_anzahl_gewaehlte_f,differenz_anzahl_gewaehlte_m,fiktive_waehlende,partei_staerke,letzte_wahl_partei_staerke,differenz_partei_staerke,flag_staerkste_partei
0,2015,0,Schweiz,1,FDP,PLR,PLR,FDP,56.0,463.0,...,7,23,3,0,3,413445.315170,16.396787,15.104533,1.292254,0
1,2015,0,Schweiz,2,CVP,PDC,PPD,CVP,55.0,442.0,...,9,19,-1,0,-1,293652.367470,11.645930,12.304013,-0.658082,0
2,2015,0,Schweiz,3,SP,PS,PS,SP,57.0,482.0,...,21,25,-3,4,-7,475074.396680,18.840929,18.722161,0.118767,0
3,2015,0,Schweiz,4,SVP,UDC,UDC,SVP,53.0,433.0,...,6,48,11,5,6,740966.800710,29.385930,26.556254,2.829675,1
4,2015,0,Schweiz,7,EVP,PEV,PEV,EVP,15.0,214.0,...,2,0,0,0,0,47770.037490,1.894507,1.997398,-0.102891,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2015,26,Jura,4,SVP,UDC,UDC,SVP,1.0,2.0,...,0,0,0,0,0,3573.490789,12.846890,15.466406,-2.619516,0
217,2015,26,Jura,8,CSP,PCS,PCS,CSP,1.0,2.0,...,0,0,0,0,0,1840.051055,6.615081,0.000000,6.615081,0
218,2015,26,Jura,9,PdA/Sol.,PST/Sol.,PdL/Sol.,PdA/Sol.,1.0,2.0,...,0,0,0,0,0,1051.026703,3.778497,0.000000,3.778497,0
219,2015,26,Jura,13,GPS,PES,PES,GPS,2.0,4.0,...,0,0,0,0,0,2023.603565,7.274962,10.965142,-3.690179,0


In [4]:
elections_2015.columns

Index(['wahl_jahr', 'kanton_nummer', 'kanton_bezeichnung', 'partei_id',
       'partei_bezeichnung_de', 'partei_bezeichnung_fr',
       'partei_bezeichnung_it', 'partei_bezeichnung_en', 'anzahl_listen',
       'anzahl_kandidierende', 'anzahl_kandidierende_f',
       'anzahl_kandidierende_m', 'anzahl_gewaehlte', 'anzahl_gewaehlte_f',
       'anzahl_gewaehlte_m', 'letzte_wahl_anzahl_gewaehlte',
       'letzte_wahl_anzahl_gewaehlte_f', 'letzte_wahl_anzahl_gewaehlte_m',
       'differenz_anzahl_gewaehlte', 'differenz_anzahl_gewaehlte_f',
       'differenz_anzahl_gewaehlte_m', 'fiktive_waehlende', 'partei_staerke',
       'letzte_wahl_partei_staerke', 'differenz_partei_staerke',
       'flag_staerkste_partei'],
      dtype='object')

In [5]:
# We keep only the relevant columns
elections_2015 = elections_2015[columns_required.keys()]
elections_2015

,wahl_jahr,kanton_bezeichnung,partei_staerke,partei_bezeichnung_de,partei_bezeichnung_fr,partei_bezeichnung_en,fiktive_waehlende
0,2015,Schweiz,16.396787,FDP,PLR,FDP,413445.315170
1,2015,Schweiz,11.645930,CVP,PDC,CVP,293652.367470
2,2015,Schweiz,18.840929,SP,PS,SP,475074.396680
3,2015,Schweiz,29.385930,SVP,UDC,SVP,740966.800710
4,2015,Schweiz,1.894507,EVP,PEV,EVP,47770.037490
...,...,...,...,...,...,...,...
216,2015,Jura,12.846890,SVP,UDC,SVP,3573.490789
217,2015,Jura,6.615081,CSP,PCS,CSP,1840.051055
218,2015,Jura,3.778497,PdA/Sol.,PST/Sol.,PdA/Sol.,1051.026703
219,2015,Jura,7.274962,GPS,PES,GPS,2023.603565


In [6]:
elections_2019 = pd.read_csv(PATH_ELECTIONS_2019, sep=';')
elections_2019

,wahl_jahr,kanton_nummer,kanton_bezeichnung,partei_id,partei_bezeichnung_de,partei_bezeichnung_fr,partei_bezeichnung_it,partei_bezeichnung_en,anzahl_listen,anzahl_kandidierende,...,differenz_anzahl_gewaehlte_f,differenz_anzahl_gewaehlte_m,fiktive_waehlende,letzte_wahl_fiktive_waehlende,differenz_fiktive_waehlende,partei_staerke,letzte_wahl_partei_staerke,differenz_partei_staerke,flag_staerkste_partei,partei_rang
0,2019,0,Schweiz,1,FDP,PLR,PLR,FDP,64.0,523.0,...,3,-7,366302.636190,413445.315170,-47143,15.110236,16.396787,-1.286551,0.0,3.0
1,2019,0,Schweiz,2,CVP,PDC,PPD,CVP,77.0,702.0,...,-2,0,275842.090500,293652.367470,-17810,11.378676,11.645930,-0.267254,0.0,5.0
2,2019,0,Schweiz,3,SP,PS,PS,SP,76.0,604.0,...,0,-4,408128.086770,475074.396680,-66946,16.835564,18.840929,-2.005364,0.0,2.0
3,2019,0,Schweiz,4,SVP,UDC,UDC,SVP,67.0,569.0,...,2,-14,620342.797060,740966.800710,-120624,25.589567,29.385930,-3.796362,1.0,1.0
4,2019,0,Schweiz,7,EVP,PEV,PEV,EVP,23.0,306.0,...,0,1,50317.220712,47770.037490,2547,2.075620,1.894507,0.181113,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2019,26,Jura,7,EVP,PEV,PEV,EVP,1.0,2.0,...,0,0,322.839682,NaN,323,1.435353,NaN,1.435353,0.0,7.0
239,2019,26,Jura,8,CSP,PCS,PCS,CSP,3.0,6.0,...,0,0,2165.632648,1840.051055,326,9.628457,6.615081,3.013376,0.0,5.0
240,2019,26,Jura,9,PdA/Sol.,PST/Sol.,PdL/Sol.,PdA/Sol.,NaN,NaN,...,0,0,NaN,1051.026703,-1051,NaN,3.778497,-3.778497,NaN,NaN
241,2019,26,Jura,13,GPS,PES,PES,GPS,2.0,4.0,...,0,0,3514.140014,2023.603565,1491,15.623955,7.274962,8.348993,0.0,3.0


In [7]:
elections_2019.columns

Index(['wahl_jahr', 'kanton_nummer', 'kanton_bezeichnung', 'partei_id',
       'partei_bezeichnung_de', 'partei_bezeichnung_fr',
       'partei_bezeichnung_it', 'partei_bezeichnung_en', 'anzahl_listen',
       'anzahl_kandidierende', 'anzahl_kandidierende_f',
       'anzahl_kandidierende_m', 'anzahl_gewaehlte', 'anzahl_gewaehlte_f',
       'anzahl_gewaehlte_m', 'letzte_wahl_anzahl_gewaehlte',
       'letzte_wahl_anzahl_gewaehlte_f', 'letzte_wahl_anzahl_gewaehlte_m',
       'differenz_anzahl_gewaehlte', 'differenz_anzahl_gewaehlte_f',
       'differenz_anzahl_gewaehlte_m', 'fiktive_waehlende',
       'letzte_wahl_fiktive_waehlende', 'differenz_fiktive_waehlende',
       'partei_staerke', 'letzte_wahl_partei_staerke',
       'differenz_partei_staerke', 'flag_staerkste_partei', 'partei_rang'],
      dtype='object')

In [8]:
# We keep only the relevant columns
elections_2019 = elections_2019[columns_required.keys()]
elections_2019

,wahl_jahr,kanton_bezeichnung,partei_staerke,partei_bezeichnung_de,partei_bezeichnung_fr,partei_bezeichnung_en,fiktive_waehlende
0,2019,Schweiz,15.110236,FDP,PLR,FDP,366302.636190
1,2019,Schweiz,11.378676,CVP,PDC,CVP,275842.090500
2,2019,Schweiz,16.835564,SP,PS,SP,408128.086770
3,2019,Schweiz,25.589567,SVP,UDC,SVP,620342.797060
4,2019,Schweiz,2.075620,EVP,PEV,EVP,50317.220712
...,...,...,...,...,...,...,...
238,2019,Jura,1.435353,EVP,PEV,EVP,322.839682
239,2019,Jura,9.628457,CSP,PCS,CSP,2165.632648
240,2019,Jura,NaN,PdA/Sol.,PST/Sol.,PdA/Sol.,NaN
241,2019,Jura,15.623955,GPS,PES,GPS,3514.140014


In [9]:
# We merge elections 2015 and 2019 together and change language to english
elections = pd.concat([elections_2015, elections_2019])
elections = elections.rename(columns=columns_required)
elections = elections.loc[elections["party's name in English"].isin(selected_parties)]
elections = elections.sort_values(by='party strength',ascending=False)
elections.head()

,year,canton,party strength,party's name in German,party's name in French,party's name in English,fictional voter
61,2015,Nidwalden,82.817529,SVP,UDC,SVP,13380.0
123,2015,Appenzell Innerrhoden,76.308068,CVP,PDC,CVP,3121.0
69,2019,Nidwalden,64.204016,SVP,UDC,SVP,9655.0
140,2019,Appenzell Innerrhoden,61.271472,CVP,PDC,CVP,3460.0
138,2019,Appenzell Ausserrhoden,49.455477,SVP,UDC,SVP,7720.0


We distinguish the results of each party from the whole Switzerland, the french-speaking part and the german-part.

In [10]:
# We select the results from the whole Switzerland
swiss_elections = elections[elections['canton'] == 'Schweiz']
swiss_elections_2015 = swiss_elections.loc[swiss_elections['year'] == 2015]
swiss_elections_2019 = swiss_elections.loc[swiss_elections['year'] == 2019]
swiss_elections.head()

,year,canton,party strength,party's name in German,party's name in French,party's name in English,fictional voter
3,2015,Schweiz,29.385930,SVP,UDC,SVP,740966.80071
3,2019,Schweiz,25.589567,SVP,UDC,SVP,620342.79706
2,2015,Schweiz,18.840929,SP,PS,SP,475074.39668
2,2019,Schweiz,16.835564,SP,PS,SP,408128.08677
0,2015,Schweiz,16.396787,FDP,PLR,FDP,413445.31517


In [11]:
# We select the results from the french-speaking part of Switzerland
romand_elections = elections.loc[elections['canton'].isin(romand_cantons)]
romand_elections_2015 = romand_elections.loc[romand_elections['year'] == 2015]
romand_elections_2019 = romand_elections.loc[romand_elections['year'] == 2019]
romand_elections.head()

,year,canton,party strength,party's name in German,party's name in French,party's name in English,fictional voter
186,2015,Valais / Wallis,39.789846,CVP,PDC,CVP,50035.731565
206,2019,Valais / Wallis,34.784426,CVP,PDC,CVP,41364.248057
214,2015,Jura,27.631841,CVP,PDC,CVP,7686.072786
236,2019,Jura,27.008715,SP,PS,SP,6074.800098
173,2015,Vaud,26.836495,FDP,PLR,FDP,48358.289662


We aggregate the strength of each of the parties proportionnaly to the population that vote for them.

In [12]:
# Sum the fictionals voters of romand cantons for each party
romand_voters_2015 = romand_elections_2015[["party's name in English",'fictional voter']].groupby("party's name in English").sum()
romand_voters_2019 = romand_elections_2019[["party's name in English",'fictional voter']].groupby("party's name in English").sum()
romand_voters_2019

,fictional voter
party's name in English,
CVP,75977.884376
FDP,105674.328188
GLP,30445.144555
GPS,96785.210955
SP,101987.077991
SVP,95498.579716


In [13]:
# We select the results from the german-speaking part of Switzerland
alemannic_elections = elections.loc[~elections['canton'].isin(romand_cantons)]
alemannic_elections_2015 = alemannic_elections.loc[alemannic_elections['year'] == 2015]
alemannic_elections_2019 = alemannic_elections.loc[alemannic_elections['year'] == 2019]
alemannic_elections.head()

,year,canton,party strength,party's name in German,party's name in French,party's name in English,fictional voter
61,2015,Nidwalden,82.817529,SVP,UDC,SVP,13380.0
123,2015,Appenzell Innerrhoden,76.308068,CVP,PDC,CVP,3121.0
69,2019,Nidwalden,64.204016,SVP,UDC,SVP,9655.0
140,2019,Appenzell Innerrhoden,61.271472,CVP,PDC,CVP,3460.0
138,2019,Appenzell Ausserrhoden,49.455477,SVP,UDC,SVP,7720.0


In [14]:
# Sum the fictionals voters of alemannic cantons for each party
alemannic_voters_2015 = alemannic_elections_2015[["party's name in English",'fictional voter']].groupby("party's name in English").sum()
alemannic_voters_2019 = alemannic_elections_2019[["party's name in English",'fictional voter']].groupby("party's name in English").sum()
alemannic_voters_2019

,fictional voter
party's name in English,
CVP,4.757063e+05
FDP,6.186967e+05
GLP,3.478798e+05
GPS,5.431917e+05
SP,7.142691e+05
SVP,1.145187e+06


Then, we have everything we need regarding the elections: the strength of the parties in the whole Switzerland, the french-speaking part and the german-speaking part for the years 2015 and 2019.

## Retrieving data from the Twitter API

The aim of this part is to gather all possible tweets from major political parties in Switzerland and from the eminent members of those parties. We selected the 6 parties with the most representatives in the Swiss National Council since the last [2019 elections](https://en.wikipedia.org/wiki/2019_Swiss_federal_election) which are the UDC, PS, PLR, Les Verts, PDC and les Verts'libéraux  in our analysis. 

To get data from the Twitter API, one needs to create a developper account. We initialize the credentials obtained from Twitter below.

In [15]:
consumer_key_1 = "92AFPa7uaAAjwxBWTfpzsc9Gc" 
consumer_secret_1 = "i7KDx59hX2nhc1vCRxewAT4woaAVlW2MJ5CddZsCFRKpxzQIw5"
access_key_1 = "2934954767-JArTI62xesl1Q5VxNXf9Mx0Czeg9vjQ7MqYufGg"
access_secret_1 = "roWovEouQhFUK7j4MewuHIKiOfKHHeG0o6OzLh5jwPfbX"

We retrieve the Twitter accounts in both languages of the 6 major political parties in Switzerland. Note that all parties except PDC have seperate Twitter accounts for communucation in French and German. PDC does all its' Twitter communication in both languages from the same account.

In [16]:
# Twitter usernames of the 6 major political parties
frenchAccounts = ['UDCch', 'PSSuisse', 'PLR_Suisse', 'LesVertsSuisses', 'vertliberaux']
germanAccounts = ['SVPch', 'spschweiz', 'FDP_Liberalen', 'GrueneCH', 'grunliberale']
germanAndFrench = ['CVP_PDC']
users = frenchAccounts + germanAccounts + germanAndFrench

The function below gets all tweets from a given list of users. The Twitter API limits us to a hard limit of a maximum of 3200 tweets per user. We try to get as many tweets as possible from each of the accounts. The features we gather are the following: 

* ``id``: The ID of a tweet 
* ``timestamp`` : The time the tweet was published
* ``partyname`` : The name of the party as defined in the twitter account
* ``username`` : The actual unique twitter username 
* ``tweet_text`` : The content of the tweet
* ``all_hashtags`` : A list containing all hashtags included in the tweet
* ``all_mentions`` : A list containing all user mentions in the tweet
* ``all_urls`` : A list containing all the URLs in the tweet
* ``retweet_count`` : The number of retweets of the given tweet
* ``favorite_count`` : The number of favorites of the given tweet
* ``range`` : The number of characters in the tweet
* `lang` : The language of the tweet


In [17]:
# Function that retrieves the maximum possible number of tweets from given accounts into a csv file.
def get_party_tweets(users, consumer_key, consumer_secret, access_key, access_secret):
    
    # Authenticate to the Twitter API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth,  parser=tweepy.parsers.JSONParser()) 
    
    # Open the CSV file to write to.
    with open('data/twitter_data/party_tweets.csv', 'w', encoding="utf-8") as file:

        w = csv.writer(file)

        # write Header row to spreadsheet
        w.writerow(['id', 'timestamp', 'partyname', 'username', 'tweet_text', 'all_hashtags', 'all_mentions', 'all_urls', 'retweet_count', 'favorite_count', 'range', 'lang'])
        
        # iterate over all usernames
        for username in users:
            # The index i is for the internal page indexing used in the Twitter API
            for i in range(18):
                # 200 is the maximum number of tweets one can retrieve per page: 200*18 = 3600 (above 3200 just to be sure)
                tweets = api.user_timeline(screen_name=username, count = 200, tweet_mode="extended", page = i)
                # Write the attributes in the CSV
                for tweet in tweets:
                    w.writerow([tweet['id'],
                                tweet['created_at'],
                                tweet['user']['name'],
                                tweet['user']['screen_name'], 
                                tweet['full_text'].replace('\n',' '),
                                [e['text'] for e in tweet['entities']['hashtags']],
                                [e['screen_name'] for e in tweet['entities']['user_mentions']],
                                [e['expanded_url'] for e in tweet['entities']['urls']],
                                tweet['retweet_count'],
                                tweet['favorite_count'],
                                tweet['display_text_range'][1],
                                tweet['lang']])

In [18]:
get_party_tweets(users, consumer_key_1, consumer_secret_1, access_key_1, access_secret_1)

Here we have the account names for all eminent members of the given parties.

In [19]:
PS_accounts = ['ChristianLevrat', 'NordmannRoger', 'beat_jans', 'jcschwaab', 'MarinaCarobbio', 'PascaleBruderer', 'SusanneSlo', 'enussbi', 'cedricwermuth', 'PaulRechsteiner', 'BaGysi', 'CarloSommaruga', 'danieljositsch', 'yferi', 'zanettiroberto', 'eviallemann', 'margretkiener', 'JayBadran', 'MathiasReynard', 'ada_marra']
PDC_accounts = ['SchmidFederer', 'ybuttet', 'ClaudeBegle', 'LeoMuellerLU', 'MarcoRomanoPPD', 'BulliardMarbach', 'fregazzi', 'Ch_Lohr', 'PirminBischof', 'gerhardpfister', 'lombardi1956', 'GraberKonrad', 'Violapamherd', 'RuthHumbel', 'MullerAltermatt', 'martin_candinas', 'Elisabeth_S_S', 'KathyRiklin', 'DdeBuman', 'engler_stefan']  
UDC_accounts = ['thomas_aeschi', 'AdrianAmstutz', 'BrandHeinz', 'jfrime', 'AmaudruzCeline', 'NatalieRickli', 'UGiezendanner', 'verenaherzog', 'SVPBrunner', 'lukasreimann', 'LuziStamm']
Les_Verts_accounts = ['bglaettli', 'RobertCramer_GE', 'SibelArslanBS', 'bastiengirod', 'RegulaRytz', 'nr_mayagraf', 'adelethorens', 'Chrige_Haesler', 'FrickerJonas']
Les_Verts_liberaux_accounts = ['tiana_moser', 'Martin_Baeumle', 'kathrinbertschy', 'beatflach', 'Juerg_Grossen', 'I_Chevalley']
PLR_accounts = ['nantermod', 'OliFrancais', 'CLuscher', 'ChristaMarkwald', 'IsabelleMoret', 'DorisFiala', 'RaphaelComteCE', 'SchneeDani67', 'fderder', 'ThierryBurkart', 'Marcel_Dobler', 'LaurentWehrli', 'Damian_Mueller_', 'FluriKurt', 'ignaziocassis', 'cwasi', 'PetraGoessi', 'RuediNoser']
all_member_accounts = PS_accounts + PDC_accounts + UDC_accounts + Les_Verts_accounts + Les_Verts_liberaux_accounts + PLR_accounts

In [20]:
# This is a helper function for us to identify the party name of the member
def get_party_name(username):
    if username in PS_accounts:
        return 'PS Suisse'
    elif username in PDC_accounts:
        return 'CVP PDC PPD PCD'
    elif username in UDC_accounts:
        return 'UDC Suisse'
    elif username in Les_Verts_accounts:
        return 'Les VERTS suisses 🌻'
    elif username in Les_Verts_liberaux_accounts:
        return "Vert'libéraux Suisse"
    elif username in PLR_accounts:
        return 'PLR Suisse'

In [21]:
# Function that retrieves the tweets of party members
def get_member_tweets(users, consumer_key, consumer_secret, access_key, access_secret, file_name):
    
    # Authenticate to the Twitter API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth,  parser=tweepy.parsers.JSONParser()) 
    
    # Open the CSV file to write to.
    with open(file_name, 'w', encoding="utf-8") as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['id', 'timestamp', 'member_name', 'party_name', 'username', 'tweet_text', 'all_hashtags', 'all_mentions', 'all_urls', 'retweet_count', 'favorite_count', 'range', 'lang'])
        
        # iterate over all usernames
        for username in users:
            
            # Get the name of the members party
            party_name = get_party_name(username)
            
            for i in range(18):
                tweets = api.user_timeline(screen_name=username, count = 200, tweet_mode="extended", page = i)
                for tweet in tweets:
                    w.writerow([tweet['id'],
                                tweet['created_at'],
                                tweet['user']['name'],
                                party_name,
                                tweet['user']['screen_name'], 
                                tweet['full_text'].replace('\n',' '),
                                [e['text'] for e in tweet['entities']['hashtags']],
                                [e['screen_name'] for e in tweet['entities']['user_mentions']],
                                [e['expanded_url'] for e in tweet['entities']['urls']],
                                tweet['retweet_count'],
                                tweet['favorite_count'],
                                tweet['display_text_range'][1],
                                tweet['lang']])

The execution of the next line in order to the tweets of all members gives an error. This is normal since we retreive too many tweets. Twitter has a limit of the number of requests one can send in an hour. We just query with the remainder of the usernames and join the CSV's obtained.

In [22]:
get_member_tweets(all_member_accounts, consumer_key_1, consumer_secret_1, access_key_1, access_secret_1, 'data/twitter_data/member_tweets.csv' )

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

We look at our CSV file and see that we exceeded the rate limit for the tweets of 'DdeBuman'. We join the rest of the parties accounts to the members of PDC for which the retrieved tweets were incomplete and launch another query.

In [23]:
member_tweets_batch_2 = ['DdeBuman', 'engler_stefan'] + UDC_accounts + Les_Verts_accounts + Les_Verts_liberaux_accounts + PLR_accounts

In [24]:
get_member_tweets(member_tweets_batch_2, consumer_key_1, consumer_secret_1, access_key_1, access_secret_1, 'data/twitter_data/member_tweets_2.csv')

No error in the execution this time. We got the maximum number of tweets from all the accounts. Now we merge both member tweet files  `member_tweets_2.csv` and `member_tweets_2.csv` into a single one by putting them into a pandas DataFrame and deleting duplicate rows.

In [25]:
df1=pd.read_csv("data/twitter_data/member_tweets.csv")
df2=pd.read_csv("data/twitter_data/member_tweets_2.csv")

full_df = pd.concat([df1,df2])
unique_df = full_df.drop_duplicates(keep='last')
unique_df.to_csv('data/twitter_data/merged_member_tweets.csv', index=False)

Finally we remove the additional CSV files we created.

In [26]:
os.remove('data/twitter_data/member_tweets.csv')
os.remove('data/twitter_data/member_tweets_2.csv')

In [27]:
party_tweets = pd.read_csv("data/twitter_data/party_tweets.csv")
member_tweets = pd.read_csv("data/twitter_data/merged_member_tweets.csv")

## RTS Data 

### I. RTS Data extraction

RTS or Radio Télévision Suisse is the french language news source for Switzerland and features various different radio and television broadcasts. The meta data for these broadcasts since 1931 archived and available through the RTS API. This meta data includes summaries of the programs and speech to text of the broadcasts audio. We want to look at the interviews done with different party members at different points in time to see which topics they are discussing and potentially their sentiment about these topics.

In [30]:
# path to the datasets
BASE_PATH ='data/RTS_dataset'
AUTHORIZATION = 'WWNNR3l4Wmh0UnY3bDNIc3R2QkhyTVo3eFVHWUVGYzE6MmplUjNpVHRranNNM2ZsWA=='
RESOURCE_URL = "https://api.srgssr.ch/rts-archives/v3/broadcasts"

The RTS API uses an OAuth security to eliminate the need for a user to disclose their user id and secret. The OAuth service acts as an inbetween between the user and resource server and grants permission with a temporary access token that must be used in the same session as the authentication took place.  

In [31]:
def get_data(payload):
    
    headers = {
    'Authorization': AUTHORIZATION,
    'Cache-Control': 'no-cache',
    'Content-Length': '0',
    'Postman-Token': '24264e32-2de0-f1e3-f3f8-eab014bb6d76'
    }
    
    while True:
        conn = http.client.HTTPSConnection("api.srgssr.ch")
        url = "https://api.srgssr.ch/oauth/v1/accesstoken?grant_type=client_credentials"
        conn.request("POST",url, "", headers)
        res = conn.getresponse()
        data = res.read()
        decode_data = json.loads(data.decode("utf-8"))
        payload['Authorization'] = "Bearer " + decode_data['access_token']

        url = "/rts-archives/v3/broadcasts/?query={}&minPublicationDate={}&maxPublicationDate={}&start={}".format(
                                                                                                        payload['query'],
                                                                                                        payload['maxPublicationDate'],
                                                                                                        payload['minPublicationDate'],
                                                                                                        payload['start'])

        conn.request("GET", url, "", payload)
        res = conn.getresponse()
        data = res.read()
        decode_data = json.loads(data.decode("utf-8"))
        if 'code' not in decode_data:
            break
    
    return decode_data, payload

In [32]:
def build_dict(query = '', minPublicationDate = '', maxPublicationDate = '',minDurationSec = '',
               maxDurationSec = '', mediaTypes = '', enumeratedFacets = '', publicationDateIntervalFacets = '',
               durationSecIntervalFacets = '', start = '', rows=''):
    
    dict_ = {
             'accept': "application/json",
             'query' : query,
             'minPublicationDate' : minPublicationDate,
             'maxPublicationDate' : maxPublicationDate,
             'minDurationSec' : minDurationSec,
             'maxDurationSec' : maxDurationSec,
             'mediaTypes' : mediaTypes,
             'enumeratedFacets' : enumeratedFacets,
             'publicationDateIntervalFacets': publicationDateIntervalFacets,
             'durationSecIntervalFacets' : durationSecIntervalFacets,
             'start' : '0',
             'Postman-Token' : '56128353-805e-4974-6689-5ef6d86e2d80',
             'rows' : rows}

    return {k: v for k,v in dict_.items() if v}

In [33]:
def request_data_and_write(payload, path=''):

    data, payload = get_data(payload)
    number_documents = data['meta']['count']
        
    if number_documents <= 0:
        return
    number_rows = 25
    
    print("number of documents:{}".format(number_documents))
    
    if 'rows' in payload and payload['rows'] > 0:
        number_rows = payload['rows']
        
    number_requests = (number_documents + number_rows // 2) // number_rows
    
    if not os.path.exists(BASE_PATH + path):
        os.makedirs(BASE_PATH + path)

    with open(BASE_PATH + path + 'data_0.json', 'w') as outfile:
        json.dump(data, outfile)
    
    for n in range(1,number_requests):
        
        #Update payload
        payload['start'] = n * number_rows

        data = get_data(payload)[0]
        
        with open(BASE_PATH + path + 'data_{}.json'.format(n), 'w') as outfile:
            json.dump(data, outfile)

In [34]:
def request_data_per_year_and_party(year, party):
    print('requesting for year:{} and party:{}'.format(year, party))
    payload = build_dict(minPublicationDate='{}'.format(year), maxPublicationDate='{}'.format(year), query ='{}'.format(party))
    request_data_and_write(payload, '_per_year/{}/{}/'.format(year, party))

In [35]:
parties = ['UDC', 'PDC', 'PS', 'PLR', 'PES', 'PVL']

#After the initial run, it was difficult to find PES and PVL data as they are normally referred to as "les Verts" or "les Verts libéraux"
#So all data was gathered for the query "Verts" and will later be filtered for irrelevant data, PES or PVL
parties.append('Verts')
years = np.arange(2012, 2020)   
for year in years:
    for party in parties:
        request_data_per_year_and_party(year, party)

requesting for year:2012 and party:Verts
number of documents:210
requesting for year:2013 and party:Verts
number of documents:179
requesting for year:2014 and party:Verts
number of documents:129
requesting for year:2015 and party:Verts
number of documents:532
requesting for year:2016 and party:Verts
number of documents:1077
requesting for year:2017 and party:Verts
number of documents:1194
requesting for year:2018 and party:Verts
number of documents:1338
requesting for year:2019 and party:Verts
number of documents:1645


For each year between 2012 and 2019 and for each of the parties listed the RTS archives API was queried for any items that were relevant to the party names, including the generic "verts" which would be used later to add to the lists of PES and PVL as it was noted that RTS does not normally refer directly to the party abbreviations. 

These queries were returned in batches of 25 documents that were then stored in the RTS_dataset_per_year folder by year and then query.

### II. RTS Data Preprocessing

In [36]:
BASE_PATH ='data/RTS_dataset_per_year/{}/{}'
parties = ['UDC', 'PDC', 'PS', 'PLR', 'PES', 'PVL']
years = np.arange(2012, 2020)

In [37]:
#Next, we want to go year by year and find interviews that are with interviews of political parties
def read_in(party):
    party_dict =[]
    
    for year in years:
        if os.path.exists(BASE_PATH.format(year, party)):
            files = [f for f in os.listdir(BASE_PATH.format(year, party))]
            for file in files:
                with open(BASE_PATH.format(year, party)+"/{}".format(file), 'r') as fp:
                    dicts = json.load(fp)
                    for entry in dicts['data']:
                        if 'genres' in entry:
                            if 'Interview' in entry['genres']:
                                party_dict.append(dicts['data'])
    return party_dict

#For each party, convert the json into dictionaries

UDC_dictionary = read_in('UDC')
PDC_dictionary = read_in('PDC')
PS_dictionary = read_in('PS')
PLR_dictionary = read_in('PLR')
PES_dictionary = read_in('PES')
PVL_dictionary = read_in('PVL')

Above, the json created by querying the RTS archive API for each of the party abbreviations was created into lists of dictionaries. However, it was noted that RTS does not often use the abbreviations for the PES and PVL parties and prefers to use the Verts and Verts libéreaux. Another query was made using the key word "verts" to find more broadcasts for these parties. Below these broadcasts were sorted into either verts or verts libéraux

In [38]:
#Below looks and sees if libéraux is in the title of an entry, if it is then the entry is sorted to the PVL dictionary

for year in years:
    if os.path.exists(BASE_PATH.format(year, 'verts')):
            files = [f for f in os.listdir(BASE_PATH.format(year, 'verts'))]
            for file in files:
                with open(BASE_PATH.format(year, 'verts')+"/{}".format(file), 'r') as fp:
                    dicts = json.load(fp)
                    for entry in dicts['data']:
                        if ('title' and 'genres') in entry:
                            if 'Interview' in entry['genres']:
                                if 'libéraux' in entry['title']:
                                    PVL_dictionary.append(entry)
                                else:
                                    PES_dictionary.append(entry)


After reading in the lists of dictionaries, each dictionary is converted to a pandas dataframe. During on our analysis we will look at either the "speechToTexts" or "summary" collumns in order to do some NLP to determine what the key issues each party were discussing and potentially their sentiment about these topics.

In [39]:
UDC_pandas = pd.DataFrame(UDC_dictionary)
PDC_pandas = pd.DataFrame(PDC_dictionary)
PS_pandas = pd.DataFrame(PS_dictionary)
PLR_pandas = pd.DataFrame(PLR_dictionary)
PES_pandas = pd.DataFrame(PES_dictionary)
PVL_pandas = pd.DataFrame(PVL_dictionary)